# IE UMAP Visualisation of 300K (3D)


UMAP with flags

## Imports

In [1]:
import sys 
import os
import pathlib
import time
print(sys.version)

3.6.9 (default, Jul  3 2019, 15:36:16) 
[GCC 5.4.0 20160609]


In [2]:
utils_path = pathlib.Path(os.getcwd() + '/utils')  # i suspect this one is not needed
print(utils_path.exists())
print(os.getcwd())
#sys.path.append(str(utils_path))  # may not be necessary
#sys.path.append(os.getcwd())  # i thnk this is the one that works 
sys.path.append('../') # this one is one level up so we can see the utils lib
print(sys.path)

True
/home/mutaz/ilab1
['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/mutaz/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/mutaz/.local/lib/python3.6/site-packages/IPython/extensions', '/home/mutaz/.ipython', '../']


In [3]:
import numpy as np
import sklearn
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from utils.data import Data
from utils.config import Config

In [4]:
import umap
import numba

## Read Data

In [5]:
d = Data()
# v2 = True: loading the dataset with the con flag
df = d.get300K_features(v2=True)

set index
time to load 8.74


In [6]:
df.head()

,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err,Atom_EA_Min,...,Atom_Ratio_dHFus_SpeedSound_StDiv,EState_Topo_Ni_Zeff,EState_Topo_Pd_Zeff,EState_Topo_Te_dHAtom,Moment_SpeedSound_1,Moment_TCurie_1,SG187,commensurate,dft_ie,dft_c33
uid,,,,,,,,,,,,,,,,,,,,,
Hf3Te2_Mo1S2-T1,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184,0.05468,...,0.03793,0.0,0.0,197.0,0.78230,0.0,0,False,False,False
Hf3Te2_W1S2-T1,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663,0.05818,...,0.03793,0.0,0.0,197.0,0.78230,0.0,0,False,False,False
Cr1S2-T1_Hf3Te2,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153,0.05106,...,0.03793,0.0,0.0,197.0,2.43511,0.0,0,False,False,False
Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941,0.00526,...,0.09867,0.0,0.0,0.0,-4.56952,0.0,0,False,False,False
Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096,0.01765,...,0.07947,0.0,0.0,197.0,-5.53334,0.0,0,False,False,False


### get descriptors column names

In [7]:
feature_cols = d.getDescriptorsColumnNames_IE()

## sampling bilayers

In [8]:
n = df.shape[0]
df_features = df.sample(n=n)



# UMAP

## umap_fit function

In [9]:
# call with df_features[feature_cols]
# labels_df has two columns C33 and communsurate 

def umap_fit(df, labels_df, 
             plot_df_filename, 
             index_name='uid', 
             n_neighbors = 15, 
             n_components = 2, 
             min_dist = 0.1, 
             metric = 'euclidean'):
    
    reducer = umap.UMAP(n_neighbors=n_neighbors,
                        n_components=n_components,
                        min_dist=min_dist,
                        metric=metric,
                        random_state=50
                       )
    
    embeddings = reducer.fit_transform(df)
    embeddings_df = pd.DataFrame(embeddings, columns={'x','y'})
    label1_name = labels_df.columns[0]
    label2_name = labels_df.columns[1]
    label3_name = labels_df.columns[2]
    label4_name = labels_df.columns[3]
    
    plot_df = pd.concat([embeddings_df, labels_df], 
              axis=1, 
              ignore_index=False)
    
    plot_df.set_index(index_name, inplace=True)
    
    plot_df.to_csv(Config().get_datapath(plot_df_filename))
    
    return(plot_df)

### Test with a small sample

In [10]:
n = 10000
neighbors = 40
components = 3

df_features = df.sample(n=n)
filename_pattern = 'umap_300Kdf_{}_{}_{}_{}_flags.csv'
filename = filename_pattern.format('IE',n,neighbors, components)
plot_df = umap_fit(df=df_features[feature_cols],
                   labels_df=df_features[['IE','commensurate', 'dft_ie', 'dft_c33']].reset_index(), 
                   plot_df_filename=filename,
                   n_neighbors=neighbors, 
                   n_components=components)

/home/mutaz/.local/lib/python3.6/site-packages/numba/typed_passes.py:271: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../.local/lib/python3.6/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  state.func_ir.loc))
/home/mutaz/.local/lib/python3.6/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../.local/lib/python3.6/site-packages/umap/utils.py", line 409:
@numba.njit(par

ValueError: Shape of passed values is (10000, 3), indices imply (10000, 2)

In [ ]:
plot_df[plot_df.dft_ie == True]

# Generate Umap plot dfs 

In [ ]:
n = df.shape[0]
neighbors_list = [40]
components = 2

df_features = df.sample(n=n)
filename_pattern = 'umap_300Kdf_{}_{}_{}_{}_flags.csv'

In [ ]:
for neighbors in neighbors_list: 
    time_start = time.time()
    filename = filename_pattern.format('IE',n,neighbors, components)
    print(f'prep for filename: {filename}')
    plot_df = umap_fit(df=df_features[feature_cols],
                   labels_df=df_features[['IE','commensurate', 'dft_ie', 'dft_ie']].reset_index(), 
                   plot_df_filename=filename,
                   n_neighbors=neighbors, 
                   n_components=components)
    print('for neighbors: {}. {:.3f} secs'.format(neighbors,time.time()-time_start))

    

In [ ]:
#umap_300Kdf_IE_10000_40_2_flags

# Plot 3d